# ETL Pipeline
Below are commands applied to construct an ETL Pipeline.
### 1. Libraries & Datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe for inspection.
- Load `categories.csv` into a dataframe for inspection.

In [1]:
# import libraries
import numpy as np
import pandas as pd
import sqlite3
import sqlalchemy
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

In [4]:
# Load datasets (messages)
messages = pd.read_csv("./Data/messages.csv")
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [ ]:
# load datasets (categories)
categories = pd.read_csv("categories.csv")
categories.head()